###Borrar y crear widgets

In [0]:
dbutils.widgets.removeAll()
dbutils.widgets.text("catalog", "bakehouse_dev")

catalogName = dbutils.widgets.get("catalog")

### Crear grupos

In [0]:
# Crear grupo
if "developers" not in [row['name'] for row in spark.sql("SHOW GROUPS").collect()]:
    spark.sql("CREATE GROUP developers")

# Adicionar miembro al grupo si no existe
member = "hugo14.gonzalez@gmail.com"
spark.sql(f"ALTER GROUP developers ADD USER `{member}`")

#members = [row['user'] for row in spark.sql("SHOW GROUP MEMBERS FROM developers").collect()]
#if member not in members:
#    spark.sql(f"ALTER GROUP developers ADD USER `{member}`")

for row in spark.sql("SHOW GROUPS").collect():
    print(row['name'])

### Asignar permisos a las tablas

In [0]:
try:
    if any(catalog.name == catalogName for catalog in spark.catalog.listCatalogs()):
        spark.sql(f"USE CATALOG `{catalogName}`")

        schemaNames = ["raw", "bronze", "silver", "gold"]
        tableNames = ["customers", "franchises", "suppliers", "transactions"]

        groups = [row['name'] for row in spark.sql("SHOW GROUPS").collect()]

        for group in groups:
            if group in ["users", "admins"]:
                continue
            #spark.sql(f"GRANT USAGE ON CATALOG `{catalogName}` TO `{group}`")
            for schemaName in schemaNames:            
                spark.sql(f"GRANT SELECT ON SCHEMA `{catalogName}`.`{schemaName}` TO `{group}`")
                if group == "developers":
                    spark.sql(f"GRANT ALL PRIVILEGES ON SCHEMA `{catalogName}`.`{schemaName}` TO `{group}`")
                else:
                    spark.sql(f"GRANT SELECT ON SCHEMA `{catalogName}`.`{schemaName}` TO `{group}`")
            for schemaName in schemaNames:
                for tableName in tableNames:
                    if group == "developers":
                        spark.sql(f"GRANT ALL PRIVILEGES ON TABLE `{catalogName}`.`{schemaName}`.`{tableName}_{schemaName}` TO `{group}`")
                    else:
                        spark.sql(f"GRANT SELECT ON TABLE `{catalogName}`.`{schemaName}`.`{tableName}_{schemaName}` TO `{group}`")
    else:
        raise Exception(f"No existe el catalog: {catalogName}")
except Exception as e:
    raise Exception(f"Error al asignar permisos en el catalogo: {catalogName}. Mensaje: {e}")

In [0]:
print("Permisos asignados")
display(spark.sql(f"SHOW GRANTS ON CATALOG `{catalogName}`"))

# Show all grants for a specific user
#display(spark.sql(f"SHOW GRANTS FOR `hugo14.gonzalez@gmail.com`"))

# Show all grants for a specific group
#display(spark.sql(f"SHOW GRANTS FOR `developers`"));

for schemaName in schemaNames:            
    display(spark.sql(f"SHOW GRANTS ON SCHEMA `{catalogName}`.`{schemaName}`"))
    #for tableName in tableNames:
    #    display(spark.sql(f"SHOW GRANTS ON TABLE `{catalogName}`.`{schemaName}`.`{tableName}_{schemaName}`"))